In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
import warnings
warnings.filterwarnings('ignore')
!pip install apyori
from apyori import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori, association_rules


In [26]:
data = pd.read_csv('Groceries-data-train.csv')
#see what data looks like
data.head(5)

,Member_number,Date,itemDescription,year,month,day,day_of_week
0,3021,30/01/2015,frankfurter,2015,1,30,4
1,1292,24/10/2015,pork,2015,10,24,5
2,4206,4/04/2014,root vegetables,2014,4,4,4
3,4369,25/08/2015,onions,2015,8,25,1
4,1522,1/07/2014,waffles,2014,7,1,1


# Task 1 Pattern Mining

In [3]:
# Group the data by 'Member_number' and 'Date', then aggregate the 'itemDescription' into lists
transactions_data = data.groupby(['Member_number'])['itemDescription'].apply(list).reset_index()

# Extract the list of transactions
transactions = transactions_data['itemDescription'].tolist()

sorted_transactions = [sorted(list(set(transaction))) for transaction in transactions]

transactions = sorted_transactions

transactions = [transaction for transaction in transactions if len(transaction) > 1]

# Display the first transaction as an example
transactions[0]

['canned beer',
 'misc. beverages',
 'pastry',
 'pickled vegetables',
 'sausage',
 'soda',
 'yogurt']

### Frequent Patterns Mining by Apriori Algorithm

In [11]:
rules = apriori(transactions, min_support=0.002, min_confidence=0.8,min_lift=1, min_length=1)
Results = list(rules)


In [15]:
df_results = pd.DataFrame(Results)
# keep support in a separate data
support = df_results.support

#all four empty list which will contain lhs, rhs, confidance and lift respectively.
first_values = []
second_values = []
third_values = []
fourth_value = []

# loop number of rows time and append 1 by 1 value in a separate list
# first and second element was frozenset which need to be converted in list
for i in range(df_results.shape[0]):
    single_list = df_results['ordered_statistics'][i][0]
    first_values.append(list(single_list[0]))
    second_values.append(list(single_list[1]))
    third_values.append(single_list[2])
    fourth_value.append(single_list[3])

In [16]:
# convert all four list into dataframe for further operation
lhs = pd.DataFrame(first_values)
rhs = pd.DataFrame(second_values)

confidance=pd.DataFrame(third_values,columns=['Confidance'])

lift=pd.DataFrame(fourth_value,columns=['lift'])

In [17]:
# concat all list together in a single dataframe
df_final = pd.concat([lhs,rhs,support,confidance,lift], axis=1)
df_final
df_final.fillna(value=' ', inplace=True)
df_final.head()

,0,1,2,3,0,support,Confidance,lift
0,bottled water,canned vegetables,,,whole milk,0.002425,0.818182,2.276067
1,spices,bottled water,,,whole milk,0.002425,0.818182,2.276067
2,butter,dishes,,,whole milk,0.002156,1.000000,2.781859
3,newspapers,canned fish,,,whole milk,0.002425,0.818182,2.276067
4,napkins,white bread,,,whole milk,0.004581,0.809524,2.251981


In [18]:
#set column name
df_final.columns = ['lhs',1,2,3,'rhs','support','confidance','lift']
df_final.head()

,lhs,1,2,3,rhs,support,confidance,lift
0,bottled water,canned vegetables,,,whole milk,0.002425,0.818182,2.276067
1,spices,bottled water,,,whole milk,0.002425,0.818182,2.276067
2,butter,dishes,,,whole milk,0.002156,1.000000,2.781859
3,newspapers,canned fish,,,whole milk,0.002425,0.818182,2.276067
4,napkins,white bread,,,whole milk,0.004581,0.809524,2.251981


In [20]:
# add all three column to lhs itemset only
df_final['lhs'] = df_final['lhs'] + str(", ")  + df_final[1] + str(", ") + df_final[2] + df_final[3]

df_final['rhs'] = df_final['rhs'] + str(", ")

df_final.drop(columns=[1,2,3],inplace=True)

df_final.sort_values('lift', ascending=False).head(10)

,lhs,rhs,support,confidance,lift
27,"waffles, bottled water, whipped/sour cream , b...","tropical fruit, ,",0.002156,0.888889,5.294810
21,"bottled water, canned beer, specialty chocolat...","root vegetables, ,",0.002156,0.888889,5.051557
32,"candy, whole milk, pork , whole milk, pork","yogurt, ,",0.002156,0.888889,4.175527
77,"tropical fruit, other vegetables, whole milkbe...","yogurt, ,",0.002425,0.818182,3.843383
6,"UHT-milk, soda, frankfurter , soda, frankfurter","rolls/buns, ,",0.002156,1.000000,3.666996
59,"sausage, frankfurter, pastry , frankfurter, pa...","soda, ,",0.002425,0.818182,3.362428
35,"shopping bags, canned beer, domestic eggs , ca...","soda, ,",0.002425,0.818182,3.362428
84,"sausage, yogurt, other vegetablesfrozen vegeta...","rolls/buns, ,",0.002425,0.900000,3.300296
44,"frozen meals, whole milk, citrus fruit , whole...","rolls/buns, ,",0.002695,0.833333,3.055830
60,"frozen meals, whole milk, frozen vegetables , ...","other vegetables, ,",0.002156,0.888889,3.037446


### Prediction Functions Based on Discovered Patterns (Ready for Import to Task 2)

In [25]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.002, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.4)
rules.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Instant food products),(root vegetables),0.011318,0.175963,0.004581,0.404762,2.300263,0.002589,1.384382,0.571738
1,(Instant food products),(soda),0.011318,0.243331,0.004850,0.428571,1.761272,0.002097,1.324171,0.437176
2,(Instant food products),(whole milk),0.011318,0.359472,0.004581,0.404762,1.125991,0.000513,1.076087,0.113174
3,(UHT-milk),(whole milk),0.054702,0.359472,0.023713,0.433498,1.205929,0.004049,1.130671,0.180646
4,(artif. sweetener),(other vegetables),0.005659,0.292643,0.002425,0.428571,1.464483,0.000769,1.237874,0.318970


# Task 2 Collaborative Filtering

In [3]:
#make a liite transform to make coding easier
df = data

#see how many unique item in training file
items = df['itemDescription'].unique()
items.shape


(167,)

In [4]:
#put same-day-purchased items in one row
member_df = df.groupby(['Member_number', 'Date'], as_index=False).agg({
    'itemDescription': ', '.join,
    'year': 'first',  # Retain the year (or use any other appropriate method)
    'month': 'first',  # Retain the month
    'day': 'first',  # Retain the day
    'day_of_week': 'first'  # Retain the day of the week
})
member_df.head(5)



,Member_number,Date,itemDescription,year,month,day,day_of_week
0,1000,15/03/2015,"sausage, yogurt",2015,3,15,6
1,1000,24/06/2014,pastry,2014,6,24,1
2,1000,24/07/2015,"misc. beverages, canned beer",2015,7,24,4
3,1000,25/11/2015,sausage,2015,11,25,2
4,1000,27/05/2015,"soda, pickled vegetables",2015,5,27,2


In [5]:
#a member one row for items

# Group by Member_number and itemDescription, then count the occurrences
purchase_counts = df.groupby(['Member_number', 'itemDescription']).size().reset_index(name='Purchase_Count')


# Initialize an empty DataFrame with Member_number as rows and itemDescription as columns
members = df['Member_number'].unique()
items = df['itemDescription'].unique()

new_df = pd.DataFrame(index=members, columns=items).fillna(0)

# Populate the new DataFrame with purchase counts
for _, row in purchase_counts.iterrows():
    member = row['Member_number']
    item = row['itemDescription']
    count = row['Purchase_Count']
    new_df.at[member, item] = count

# Reset the index to make Member_number a column
new_df = new_df.reset_index().rename(columns={'index': 'Member_number'})



In [6]:
#sort by member-number
new_df.set_index('Member_number', inplace=True)
new_df = new_df.sort_index()
new_df.shape
new_df.head(5)

,frankfurter,pork,root vegetables,onions,waffles,cereals,yogurt,sausage,rolls/buns,chocolate,...,organic sausage,house keeping products,frozen fruits,bathroom cleaner,skin care,bags,rubbing alcohol,make up remover,preservation products,kitchen utensil
Member_number,,,,,,,,,,,,,,,,,,,,,
1000,0,0,0,0,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1001,1,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1003,0,0,1,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1004,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


# TF-IDF
Using tf-idf method to calculate how important an item for a member to see wether we can get better result

In [7]:
#calculate TF
tf = new_df.copy()
# Count the number of 0s in each row
tf = tf.apply(lambda row: row / (167 - (row == 0).sum()), axis=1)


In [8]:
#caluculate IDF
columns = tf.columns
idf = []
for i in columns:
  idf.append(np.log10(3872 / (tf[i]!=0).sum())) # number of member / number of member purchse this item

In [9]:
#combine TF and IDF
tf_idf = tf.apply(lambda row: row * idf, axis=1)
tf_idf.head(5)

,frankfurter,pork,root vegetables,onions,waffles,cereals,yogurt,sausage,rolls/buns,chocolate,...,organic sausage,house keeping products,frozen fruits,bathroom cleaner,skin care,bags,rubbing alcohol,make up remover,preservation products,kitchen utensil
Member_number,,,,,,,,,,,,,,,,,,,,,
1000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.098068,0.236864,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001,0.110394,0.0,0.000000,0.0,0.0,0.0,0.000000,0.092114,0.064418,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003,0.000000,0.0,0.154206,0.0,0.0,0.0,0.000000,0.165805,0.115952,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1004,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.044597,0.092077,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# UV decomposition

In [10]:
# build UV decomposition
def uv_decomposition(R, k, learning_rate, regularization):
    """
    Performs UV decomposition on the input matrix R, with a target rank of k, using stochastic gradient descent (SGD).
    Returns the decomposed matrices U and V.
    """
    # Initialize U and V with random values
    num_users, num_items = R.shape
    U = np.random.rand(num_users, k)
    V = np.random.rand(k, num_items)

    # Perform stochastic gradient descent to optimize U and V
    for epoch in range(10):
        for i in range(num_users):
            for j in range(num_items):
                if R[i, j] > 0:
                    error = R[i, j] - np.dot(U[i, :], V[:, j])
                    U[i, :] += learning_rate * (error * V[:, j] - regularization * U[i, :])
                    V[:, j] += learning_rate * (error * U[i, :] - regularization * V[:, j])

    # Return the decomposed matrices U and V
    return U, V

numpy_array = new_df.to_numpy()


In [11]:
#To choose best k by RMSE
def calculate_rmse(R, U, V):
    predicted_R = np.dot(U, V)
    error = R - predicted_R
    error = error[R > 0]  # Only consider known values
    return np.sqrt(np.mean(error**2))

k_values = [1,2,3,4,5,6,7,8,9]

# Split the data into training and validation sets

best_k = None
best_rmse = float('inf')

for k in k_values:
    U, V = uv_decomposition(numpy_array, k, 0.1, 0.1)
    rmse = calculate_rmse(numpy_array, U, V)
    print(f"k={k}, RMSE={rmse}")
    if rmse < best_rmse:
        best_rmse = rmse
        best_k = k

print(f"Best k: {best_k} with RMSE: {best_rmse}")

k=1, RMSE=0.2988518222752414
k=2, RMSE=0.29781974880827367
k=3, RMSE=0.29693572728061374
k=4, RMSE=0.2974634347869236
k=5, RMSE=0.2947752672453374
k=6, RMSE=0.29670984454292815
k=7, RMSE=0.2989411500382539
k=8, RMSE=0.29902241520866296
k=9, RMSE=0.30038004108279354
Best k: 5 with RMSE: 0.2947752672453374


In [12]:
# Perform UV decomposition on "raw training data"
U, V = uv_decomposition(numpy_array, k=best_k, learning_rate=0.1, regularization=0.1)

# Reconstruct the matrix using the decomposed matrices U and V
new_df_columns = new_df.columns
numpy_array_reconstructed = np.dot(U, V)
df_reconstructed = pd.DataFrame(numpy_array_reconstructed, columns=new_df_columns)
df_reconstructed.index = new_df.index

#remove the items customer already bought
assist_df = new_df.apply(lambda row: row.apply(lambda x: 0 if x != 0 else 1), axis=1)
df_reconstructed = assist_df*df_reconstructed


In [13]:
#print recommendation
a=[]
for i in range(df_reconstructed.shape[0]):
  row = df_reconstructed.iloc[i]
  top_5_values = row.nlargest(5)
  top_5_columns = top_5_values.index
  b = []
  for ii in range(5):
      b.append((top_5_columns[ii], top_5_values[ii]))
  a.append(b)
member_recomendation = pd.DataFrame(index=new_df.index, columns=["top_1", "top_2", "top_3", "top_4", "top_5"], data = a)
member_recomendation.head(5)

,top_1,top_2,top_3,top_4,top_5
Member_number,,,,,
1000,"(margarine, 1.2908488082880267)","(other vegetables, 1.2107035942257451)","(rolls/buns, 1.1249168037991504)","(pip fruit, 1.1082472694345398)","(soft cheese, 1.0991622127913927)"
1001,"(margarine, 1.4444500253551433)","(other vegetables, 1.360337067547286)","(yogurt, 1.3494821821607943)","(pip fruit, 1.261738958325996)","(kitchen utensil, 1.2430595038353527)"
1002,"(margarine, 1.1780134515597867)","(other vegetables, 1.1254642697838515)","(yogurt, 1.106977665638572)","(sausage, 1.083905242299408)","(rolls/buns, 1.0645067556721812)"
1003,"(margarine, 1.1101066713612728)","(other vegetables, 1.0690643691952635)","(bags, 1.0685489655463907)","(yogurt, 1.0425896991457777)","(pip fruit, 1.0039831637974503)"
1004,"(margarine, 1.2932523944899492)","(yogurt, 1.2160551085842708)","(sausage, 1.1956079868386225)","(kitchen utensil, 1.1230831717182108)","(soft cheese, 1.1116425797972085)"


In [14]:
# Perform UV decomposition on "tf_idf training data"

tfidf_numpy_array = tf_idf.to_numpy()

U_, V_ = uv_decomposition(tfidf_numpy_array, k=best_k, learning_rate=0.1, regularization=0.1)

# Reconstruct the matrix R using the decomposed matrices U and V
tfidf_numpy_array_reconstructed = np.dot(U_, V_)
df_tfidf = pd.DataFrame(tfidf_numpy_array_reconstructed, columns=new_df_columns)
df_tfidf.index = new_df.index

#remove the items customer already bought
assist_df = new_df.apply(lambda row: row.apply(lambda x: 0 if x != 0 else 1), axis=1)
df_tfidf = assist_df*df_tfidf
df_tfidf.head(5)

,frankfurter,pork,root vegetables,onions,waffles,cereals,yogurt,sausage,rolls/buns,chocolate,...,organic sausage,house keeping products,frozen fruits,bathroom cleaner,skin care,bags,rubbing alcohol,make up remover,preservation products,kitchen utensil
Member_number,,,,,,,,,,,,,,,,,,,,,
1000,0.059769,0.095043,0.066516,0.075522,0.095394,0.103391,0.000000,0.000000,0.048105,0.068498,...,0.150196,0.109966,0.180747,0.140291,0.111792,0.104548,0.105056,0.177655,0.292144,0.258001
1001,0.000000,0.059041,0.044227,0.052026,0.069608,0.074560,0.034652,0.000000,0.000000,0.045697,...,0.112534,0.069120,0.125861,0.116020,0.077644,0.026515,0.017974,0.108363,0.131448,0.238857
1002,0.074249,0.117551,0.085088,0.095718,0.119409,0.134689,0.061106,0.091955,0.060520,0.085331,...,0.182982,0.140638,0.274851,0.184227,0.135709,0.153510,0.131109,0.205818,0.339738,0.321722
1003,0.124494,0.206518,0.000000,0.179552,0.194272,0.253357,0.119678,0.000000,0.000000,0.141916,...,0.315384,0.277229,0.747540,0.341273,0.239054,0.287816,0.236805,0.324990,0.387765,0.615695
1004,0.037807,0.057719,0.046366,0.053607,0.065893,0.070732,0.035961,0.051320,0.000000,0.000000,...,0.109308,0.082401,0.169138,0.090322,0.076533,0.076290,0.069395,0.081097,0.109554,0.215996


In [15]:
#print recommendation
z=[]
for i in range(df_tfidf.shape[0]):
  row = df_tfidf.iloc[i]
  top_5_values = row.nlargest(5)
  top_5_columns = top_5_values.index
  y = []
  for ii in range(5):
      y.append((top_5_columns[ii], top_5_values[ii]))
  z.append(y)
tfidf_member_recomendation = pd.DataFrame(index=new_df.index, columns=["top_1", "top_2", "top_3", "top_4", "top_5"], data = z)
tfidf_member_recomendation.head(5)

,top_1,top_2,top_3,top_4,top_5
Member_number,,,,,
1000,"(baby cosmetics, 0.3320926511720532)","(cream, 0.313559577465331)","(organic products, 0.31080526888586435)","(preservation products, 0.29214396013209987)","(kitchen utensil, 0.2580009056739938)"
1001,"(organic products, 0.29091925122210593)","(baby cosmetics, 0.26617507233120785)","(kitchen utensil, 0.2388574711287189)","(frozen chicken, 0.19823041784171286)","(cream, 0.189749114182209)"
1002,"(baby cosmetics, 0.3841934336963079)","(cream, 0.3727296595542674)","(organic products, 0.35880965676671883)","(preservation products, 0.33973785769088677)","(kitchen utensil, 0.32172195385831853)"
1003,"(organic products, 0.7742874229590598)","(frozen fruits, 0.747540440286308)","(cooking chocolate, 0.6611339485162511)","(kitchen utensil, 0.6156948963168777)","(baby cosmetics, 0.5856306248227824)"
1004,"(organic products, 0.2546879539343749)","(cooking chocolate, 0.23556736708670856)","(kitchen utensil, 0.2159959488661519)","(baby cosmetics, 0.19765818575876773)","(frozen fruits, 0.1691377863553393)"


# Measure the performance.

In [16]:
df_test = pd.read_csv('Groceries-data-test.csv')
df_test.head(5)

,Member_number,Date,itemDescription,year,month,day,day_of_week
0,3481,8/03/2015,candy,2015,3,8,6
1,1254,19/04/2015,white wine,2015,4,19,6
2,2835,28/01/2014,domestic eggs,2014,1,28,1
3,2854,2/08/2015,coffee,2015,8,2,6
4,4637,12/08/2014,bottled water,2014,8,12,1


In [17]:
#a member one row for items

# Group by Member_number and itemDescription, then count the occurrences
purchase_counts = df_test.groupby(['Member_number', 'itemDescription']).size().reset_index(name='Purchase_Count')


# Initialize an empty DataFrame with Member_number as rows and itemDescription as columns
members = df_test['Member_number'].unique()
items = df_test['itemDescription'].unique()

new_df_test = pd.DataFrame(index=members, columns=items).fillna(0)

# Populate the new DataFrame with purchase counts
for _, row in purchase_counts.iterrows():
    member = row['Member_number']
    item = row['itemDescription']
    count = row['Purchase_Count']
    new_df_test.at[member, item] = count

# Reset the index to make Member_number a column
new_df_test = new_df_test.reset_index().rename(columns={'index': 'Member_number'})

#sort by member-number
new_df_test.set_index('Member_number', inplace=True)
new_df_test = new_df_test.sort_index()
new_df_test.head(5)


,candy,white wine,domestic eggs,coffee,bottled water,long life bakery product,soda,frankfurter,pip fruit,tropical fruit,...,frozen chicken,make up remover,brandy,cooking chocolate,bags,canned fruit,pudding powder,potato products,soap,honey
Member_number,,,,,,,,,,,,,,,,,,,,,
1000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1001,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1004,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [18]:
#print recommendation
a=[]
for i in range(df_reconstructed.shape[0]):
  row = df_reconstructed.iloc[i]
  top_5_values = row.nlargest(5)
  top_5_columns = top_5_values.index
  b = []
  for ii in range(5):
      b.append((top_5_columns[ii], top_5_values[ii]))
  a.append(b)
member_recomendation = pd.DataFrame(index=new_df.index, columns=["top_1", "top_2", "top_3", "top_4", "top_5"], data = a)
member_recomendation.head(5)

,top_1,top_2,top_3,top_4,top_5
Member_number,,,,,
1000,"(margarine, 1.2908488082880267)","(other vegetables, 1.2107035942257451)","(rolls/buns, 1.1249168037991504)","(pip fruit, 1.1082472694345398)","(soft cheese, 1.0991622127913927)"
1001,"(margarine, 1.4444500253551433)","(other vegetables, 1.360337067547286)","(yogurt, 1.3494821821607943)","(pip fruit, 1.261738958325996)","(kitchen utensil, 1.2430595038353527)"
1002,"(margarine, 1.1780134515597867)","(other vegetables, 1.1254642697838515)","(yogurt, 1.106977665638572)","(sausage, 1.083905242299408)","(rolls/buns, 1.0645067556721812)"
1003,"(margarine, 1.1101066713612728)","(other vegetables, 1.0690643691952635)","(bags, 1.0685489655463907)","(yogurt, 1.0425896991457777)","(pip fruit, 1.0039831637974503)"
1004,"(margarine, 1.2932523944899492)","(yogurt, 1.2160551085842708)","(sausage, 1.1956079868386225)","(kitchen utensil, 1.1230831717182108)","(soft cheese, 1.1116425797972085)"


In [41]:
#calculate precise rate : if item of recomendation is in test dataset => count +=1
itemsets = df_test.groupby("Member_number")["itemDescription"].apply(list).reset_index()
x = set(itemsets.iloc[0, 1])
count = 0
for i in range(itemsets.shape[0]):
  recomendation_item = set()
  recomendation_item.add(member_recomendation.iloc[i, 0][0])
  recomendation_item.add(member_recomendation.iloc[i, 1][0])
  recomendation_item.add(member_recomendation.iloc[i, 2][0])
  recomendation_item.add(member_recomendation.iloc[i, 3][0])
  recomendation_item.add(member_recomendation.iloc[i, 4][0])
  intersection = recomendation_item.intersection(set(itemsets.iloc[i, 1]))
  if len(intersection) != 0:
    count += 1

print("precise rate:")
raw_precise = count/itemsets.shape[0]
print(raw_precise)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


precise rate:
0.3115535614133483


In [20]:
#print recommendation from tf-idf
z=[]
for i in range(df_tfidf.shape[0]):
  row = df_tfidf.iloc[i]
  top_5_values = row.nlargest(5)
  top_5_columns = top_5_values.index
  y = []
  for ii in range(5):
      y.append((top_5_columns[ii], top_5_values[ii]))
  z.append(y)
tfidf_member_recomendation = pd.DataFrame(index=new_df.index, columns=["top_1", "top_2", "top_3", "top_4", "top_5"], data = z)
tfidf_member_recomendation.head(5)

,top_1,top_2,top_3,top_4,top_5
Member_number,,,,,
1000,"(baby cosmetics, 0.3320926511720532)","(cream, 0.313559577465331)","(organic products, 0.31080526888586435)","(preservation products, 0.29214396013209987)","(kitchen utensil, 0.2580009056739938)"
1001,"(organic products, 0.29091925122210593)","(baby cosmetics, 0.26617507233120785)","(kitchen utensil, 0.2388574711287189)","(frozen chicken, 0.19823041784171286)","(cream, 0.189749114182209)"
1002,"(baby cosmetics, 0.3841934336963079)","(cream, 0.3727296595542674)","(organic products, 0.35880965676671883)","(preservation products, 0.33973785769088677)","(kitchen utensil, 0.32172195385831853)"
1003,"(organic products, 0.7742874229590598)","(frozen fruits, 0.747540440286308)","(cooking chocolate, 0.6611339485162511)","(kitchen utensil, 0.6156948963168777)","(baby cosmetics, 0.5856306248227824)"
1004,"(organic products, 0.2546879539343749)","(cooking chocolate, 0.23556736708670856)","(kitchen utensil, 0.2159959488661519)","(baby cosmetics, 0.19765818575876773)","(frozen fruits, 0.1691377863553393)"


In [44]:
#calculate precise rate : if item of recomendation is in test dataset => count +=1

itemsets = df_test.groupby("Member_number")["itemDescription"].apply(list).reset_index()
x = set(itemsets.iloc[0, 1])
count = 0
for i in range(itemsets.shape[0]):
  recomendation_item = set()
  recomendation_item.add(tfidf_member_recomendation.iloc[i, 0][0])
  recomendation_item.add(tfidf_member_recomendation.iloc[i, 1][0])
  recomendation_item.add(tfidf_member_recomendation.iloc[i, 2][0])
  recomendation_item.add(tfidf_member_recomendation.iloc[i, 3][0])
  recomendation_item.add(tfidf_member_recomendation.iloc[i, 4][0])
  intersection = recomendation_item.intersection(set(itemsets.iloc[i, 1]))
  if len(intersection) != 0:
    count += 1

print("precise rate:")
tfidf_precise = count/itemsets.shape[0]

print(tfidf_precise)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


precise rate:
0.003925967470555244


In [51]:
print(f"raw data precise rate: {raw_precise}")
print(f"tf-idf data precise rate: {tfidf_precise}")

raw data precise rate: 0.3115535614133483
tf-idf data precise rate: 0.003925967470555244


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


*Don't do tf-idf to data will get better result.*

# Make recommendation from pattern(task 1)

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth

In [24]:
#from task 1

data = pd.read_csv('Groceries-data-train.csv')

# Group the data by 'Member_number' and 'Date', then aggregate the 'itemDescription' into lists
transactions_data = data.groupby(['Member_number'])['itemDescription'].apply(list).reset_index()

# Extract the list of transactions
transactions = transactions_data['itemDescription'].tolist()

sorted_transactions = [sorted(list(set(transaction))) for transaction in transactions]

transactions = sorted_transactions

transactions = [transaction for transaction in transactions if len(transaction) > 1]

# Display the first transaction as an example
transactions[0]

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
#from task 1
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(df, min_support=0.002, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.4)
rules.head(5)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Instant food products),(root vegetables),0.011318,0.175963,0.004581,0.404762,2.300263,0.002589,1.384382,0.571738
1,(Instant food products),(soda),0.011318,0.243331,0.004850,0.428571,1.761272,0.002097,1.324171,0.437176
2,(Instant food products),(whole milk),0.011318,0.359472,0.004581,0.404762,1.125991,0.000513,1.076087,0.113174
3,(UHT-milk),(whole milk),0.054702,0.359472,0.023713,0.433498,1.205929,0.004049,1.130671,0.180646
4,(artif. sweetener),(other vegetables),0.005659,0.292643,0.002425,0.428571,1.464483,0.000769,1.237874,0.318970


In [26]:
#from task 1

def predict_items(purchased_items):
    # create a list to store the predicted items
    predicted_items = []

    # iterate over the top rules
    for index, row in rules.iterrows():
        # get the items in the antecedent and consequent of the rule
        antecedent = row["antecedents"]
        consequent = row["consequents"]

        # check if all the items in the antecedent are in the purchased items
        if antecedent.issubset(purchased_items):
            # add the items in the consequent to the predicted items
            for item in consequent:
                if item not in purchased_items and item not in predicted_items:
                    predicted_items.append(item)

    # return the list of predicted items
    return predicted_items

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
# make a prediction for a sample set of purchased items
purchased_items = {'canned beer',
 'misc. beverages',
 'pastry',
 'pickled vegetables',
 'sausage',
 'soda',
 'yogurt'}
predicted_items = predict_items(purchased_items)

# print the predicted items to the console
print(predicted_items)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['whole milk', 'rolls/buns', 'other vegetables', 'pip fruit']


In [46]:
#output recomendation set
df_origin = pd.read_csv('Groceries-data-train.csv')

pattern_recommendation = new_df.copy()
pattern_recommendation.drop(pattern_recommendation.columns, axis=1, inplace=True)

itemsets = df_origin.groupby("Member_number")["itemDescription"].apply(list).reset_index()
item_array = []
for i in range(itemsets.shape[0]):
  current_itemset = itemsets.iloc[i, 1]
  predicted_items = predict_items(current_itemset)
  item_array.append(predicted_items)
pattern_recommendation['recommendation_item'] = item_array
pattern_recommendation.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,recommendation_item
Member_number,
1000,"[whole milk, rolls/buns, other vegetables, pip..."
1001,"[other vegetables, yogurt, pip fruit]"
1002,"[rolls/buns, other vegetables, yogurt, soda]"
1003,"[whole milk, other vegetables]"
1004,"[soda, root vegetables, yogurt, bottled water,..."


In [52]:
#calculate precise rate : if item of recomendation is in test dataset => count +=1
itemsets = df_test.groupby("Member_number")["itemDescription"].apply(list).reset_index()
x = set(itemsets.iloc[0, 1])
count = 0
for i in range(itemsets.shape[0]):
  recomendation_item = set(pattern_recommendation.iloc[i, 0])
  intersection = recomendation_item.intersection(set(itemsets.iloc[i, 1]))
  if len(intersection) != 0:
    count += 1
print("precise rate:")
pattern_precise = count/itemsets.shape[0]
print(pattern_precise)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


precise rate:
0.3202467750981492


In [53]:
print(f"raw data precise rate: {raw_precise}")
print(f"tf-idf data precise rate: {tfidf_precise}")
print(f"pattern precise rate: {pattern_precise}")

raw data precise rate: 0.3115535614133483
tf-idf data precise rate: 0.003925967470555244
pattern precise rate: 0.3202467750981492


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
